In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')


In [4]:
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')


In [5]:
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [6]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


In [7]:
price_df = pd.read_csv('/content/price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [8]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
1,00114d74e963e47177db89bc70108537,-0.003994,-0.000001
2,001cd16732dc7d5bdf46b0d49996c271,0.047697,2.177779
3,001dc71143f45cb58aaccc2e94823c5a,-0.003778,0.177779
4,002b3009d069858b471918402fb237b7,-0.005795,0.000000


In [9]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

In [10]:
# Get January and December prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

In [11]:
# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]

In [12]:
# Merge the new features into the main dataframe
df = df.merge(diff, on='id', how='left')

In [14]:
#Contract Duration in Days
df['contract_duration'] = (df['date_end'] - df['date_activ']).dt.days

In [15]:
# Time Since Last Product Modification in Days
df['time_since_last_modification'] = (df['date_renewal'] - df['date_modif_prod']).dt.days

In [16]:
# Average Monthly Price Change (using available monthly price data)
monthly_avg_prices = price_df.groupby(['id', price_df['price_date'].dt.to_period('M')]).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()
monthly_avg_prices['price_date'] = monthly_avg_prices['price_date'].dt.to_timestamp()
monthly_avg_prices['monthly_price_change_var'] = monthly_avg_prices.groupby('id')['price_off_peak_var'].diff()
monthly_avg_prices['monthly_price_change_fix'] = monthly_avg_prices.groupby('id')['price_off_peak_fix'].diff()

In [17]:
# Merge average monthly price change into the main dataframe
df = df.merge(monthly_avg_prices[['id', 'price_date', 'monthly_price_change_var', 'monthly_price_change_fix']], left_on=['id', 'date_renewal'], right_on=['id', 'price_date'], how='left').drop(columns=['price_date'])

In [18]:
# Flag for Active Contracts (assuming contracts without end date are active)
df['is_active'] = df['date_end'].isna().astype(int)

In [19]:
# Contract Renewal Rate (number of renewals per year)
renewal_counts = df.groupby(df['date_renewal'].dt.year)['id'].count().reset_index(name='renewals_per_year')
df = df.merge(renewal_counts, left_on=df['date_renewal'].dt.year, right_on='date_renewal')

In [20]:
# Number of Modifications
df['num_modifications'] = df.groupby('id')['date_modif_prod'].transform('count')

In [24]:
print(df.head())

   date_renewal                                id  \
0          2015  24011ae4ebbe3035111d65fa7c15bc57   
1          2015  d29c2c54acc38ff3c0614d0a653813dd   
2          2015  764c75f661154dac3a6c254cd082ea7d   
3          2015  bba03439a292a1e166f80264c16191cb   
4          2015  149d57cf92fc41cf94415803a877cb4b   

                      channel_sales  cons_12m  cons_gas_12m  cons_last_month  \
0  foosdfpfkusacimwkcsosbicdxkicaua         0         54946                0   
1                           MISSING      4660             0                0   
2  foosdfpfkusacimwkcsosbicdxkicaua       544             0                0   
3  lmkebamcaaclubfxadlmueccxoimlema      1584             0                0   
4                           MISSING      4425             0              526   

  date_activ   date_end date_modif_prod date_renewal_x  ...  \
0 2013-06-15 2016-06-15      2015-11-01     2015-06-23  ...   
1 2009-08-21 2016-08-30      2009-08-21     2015-08-31  ...   
2 2010-04-1

In [25]:
df.to_csv('enhanced_data_with_additional_features.csv', index=False)